# I. Import_Stock_List

In [2]:
file_path = "../Servers/stock_symbol_list.txt"
stock_list = []
with open(file_path, 'r') as file:
    for line in file:
        stock_list.append(line.strip())
print(stock_list)

['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']


## II. Import 1m prices from FINHUB

In [18]:
from dotenv import load_dotenv
import polars as pl
import finnhub
import os

load_dotenv("./keys/finhub.env", override=True)
FINHUB_API = os.getenv("FINHUB_API")
FINANCIALMODELINGPREP_API = os.getenv("FINANCIALMODELINGPREP_API")
FINANCIALMODELINGPREP_PROFILE_ROUTE = os.getenv("FINANCIALMODELINGPREP_PROFILE_ROUTE")

# III. Import neccessary libraries and set up environment

### 1. Quote Real Time Single Stock Symbol Price

In [7]:
finnhub_client = finnhub.Client(api_key=FINHUB_API)
print(finnhub_client.quote("AAPL"))

{'c': 225, 'd': -3.22, 'dp': -1.4109, 'h': 226.92, 'l': 224.27, 'o': 226.4, 'pc': 228.22, 't': 1731790800}


### 2. Quote Real Time Prices for Stock List

In [12]:
symbol_prices_dict = dict()

for symbol in stock_list:
    price_data = finnhub_client.quote(symbol)
    symbol_prices_dict[symbol] = price_data
    print(symbol + " Real Time Quote")
    print(str(price_data) + "\n")

symbol_prices_dict

AAPL Real Time Quote
{'c': 225, 'd': -3.22, 'dp': -1.4109, 'h': 226.92, 'l': 224.27, 'o': 226.4, 'pc': 228.22, 't': 1731790800}

MSFT Real Time Quote
{'c': 415, 'd': -11.89, 'dp': -2.7853, 'h': 422.8, 'l': 413.64, 'o': 419.82, 'pc': 426.89, 't': 1731790800}

GOOGL Real Time Quote
{'c': 172.49, 'd': -3.09, 'dp': -1.7599, 'h': 174.14, 'l': 171.22, 'o': 173.73, 'pc': 175.58, 't': 1731790800}

AMZN Real Time Quote
{'c': 202.61, 'd': -8.87, 'dp': -4.1943, 'h': 207.34, 'l': 199.61, 'o': 206.76, 'pc': 211.48, 't': 1731790800}

TSLA Real Time Quote
{'c': 320.72, 'd': 9.54, 'dp': 3.0657, 'h': 324.6799, 'l': 309.22, 'o': 310.57, 'pc': 311.18, 't': 1731790800}



{'AAPL': {'c': 225,
  'd': -3.22,
  'dp': -1.4109,
  'h': 226.92,
  'l': 224.27,
  'o': 226.4,
  'pc': 228.22,
  't': 1731790800},
 'MSFT': {'c': 415,
  'd': -11.89,
  'dp': -2.7853,
  'h': 422.8,
  'l': 413.64,
  'o': 419.82,
  'pc': 426.89,
  't': 1731790800},
 'GOOGL': {'c': 172.49,
  'd': -3.09,
  'dp': -1.7599,
  'h': 174.14,
  'l': 171.22,
  'o': 173.73,
  'pc': 175.58,
  't': 1731790800},
 'AMZN': {'c': 202.61,
  'd': -8.87,
  'dp': -4.1943,
  'h': 207.34,
  'l': 199.61,
  'o': 206.76,
  'pc': 211.48,
  't': 1731790800},
 'TSLA': {'c': 320.72,
  'd': 9.54,
  'dp': 3.0657,
  'h': 324.6799,
  'l': 309.22,
  'o': 310.57,
  'pc': 311.18,
  't': 1731790800}}

### 3. Convert dictionary to list of rows and create polars dataframe

In [14]:
rows = [
    {
        "stock_symbol": symbol,
        "current_price": data["c"],
        "change": data["d"],
        "percent_change": data["dp"],
        "high_price_of_the_day": data["h"],
        "low_price_of_the_day": data["l"],
        "open_price_of_the_day": data["o"],
        "previous_close_price": data["pc"]
    }
    for symbol, data in symbol_prices_dict.items()
]
df = pl.DataFrame(rows)

In [15]:
df

stock_symbol,current_price,change,percent_change,high_price_of_the_day,low_price_of_the_day,open_price_of_the_day,previous_close_price
str,f64,f64,f64,f64,f64,f64,f64
"""AAPL""",225.0,-3.22,-1.4109,226.92,224.27,226.4,228.22
"""MSFT""",415.0,-11.89,-2.7853,422.8,413.64,419.82,426.89
"""GOOGL""",172.49,-3.09,-1.7599,174.14,171.22,173.73,175.58
"""AMZN""",202.61,-8.87,-4.1943,207.34,199.61,206.76,211.48
"""TSLA""",320.72,9.54,3.0657,324.6799,309.22,310.57,311.18


# IV. Retrieve Company Information

In [21]:
from urllib.request import urlopen
import certifi
import json
import ssl

def get_jsonparsed_data(url):
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    response = urlopen(url, context=ssl_context)
    data = response.read().decode("utf-8")
    return json.loads(data)

url = (FINANCIALMODELINGPREP_PROFILE_ROUTE + "AAPL" + "?apikey=" + FINANCIALMODELINGPREP_API)
print(get_jsonparsed_data(url))

[{'symbol': 'AAPL', 'price': 225, 'beta': 1.24, 'volAvg': 47719342, 'mktCap': 3401055000000, 'lastDiv': 0.99, 'range': '164.08-237.49', 'changes': -3.22, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, vide

In [22]:
symbol_company_infor_dict = dict()

for symbol in stock_list:
    url = (FINANCIALMODELINGPREP_PROFILE_ROUTE + symbol + "?apikey=" + FINANCIALMODELINGPREP_API)
    company_infor = get_jsonparsed_data(url)
    symbol_company_infor_dict[symbol] = company_infor
    print(symbol + " Company Informmation")
    print(str(company_infor) + "\n")

AAPL Company Informmation
[{'symbol': 'AAPL', 'price': 225, 'beta': 1.24, 'volAvg': 47719342, 'mktCap': 3401055000000, 'lastDiv': 0.99, 'range': '164.08-237.49', 'changes': -3.22, 'companyName': 'Apple Inc.', 'currency': 'USD', 'cik': '0000320193', 'isin': 'US0378331005', 'cusip': '037833100', 'exchange': 'NASDAQ Global Select', 'exchangeShortName': 'NASDAQ', 'industry': 'Consumer Electronics', 'website': 'https://www.apple.com', 'description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, 

In [24]:
rows_company_infor_dataframe = [data for symbol, data_list in symbol_company_infor_dict.items() for data in data_list]
df_company_infor = pl.DataFrame(rows_company_infor_dataframe)

In [25]:
df_company_infor

symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,cik,isin,cusip,exchange,exchangeShortName,industry,website,description,ceo,sector,country,fullTimeEmployees,phone,address,city,state,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
str,f64,f64,i64,i64,f64,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,str,str,bool,bool,bool,bool,bool
"""AAPL""",225.0,1.24,47719342,3401055000000,0.99,"""164.08-237.49""",-3.22,"""Apple Inc.""","""USD""","""0000320193""","""US0378331005""","""037833100""","""NASDAQ Global Select""","""NASDAQ""","""Consumer Electronics""","""https://www.apple.com""","""Apple Inc. designs, manufactur…","""Mr. Timothy D. Cook""","""Technology""","""US""","""164000""","""408 996 1010""","""One Apple Park Way""","""Cupertino""","""CA""","""95014""",76.28377,148.716225,"""https://images.financialmodeli…","""1980-12-12""",false,false,true,false,false
"""MSFT""",415.0,0.904,19210821,3085475200000,2.25,"""362.9-468.35""",-11.89,"""Microsoft Corporation""","""USD""","""0000789019""","""US5949181045""","""594918104""","""NASDAQ Global Select""","""NASDAQ""","""Software - Infrastructure""","""https://www.microsoft.com""","""Microsoft Corporation develops…","""Mr. Satya Nadella""","""Technology""","""US""","""228000""","""425 882 8080""","""One Microsoft Way""","""Redmond""","""WA""","""98052-6399""",47.70472,367.295277,"""https://images.financialmodeli…","""1986-03-13""",false,false,true,false,false
"""GOOGL""",172.49,1.034,24372353,2120041614569,0.4,"""127.9-191.75""",-3.09,"""Alphabet Inc.""","""USD""","""0001652044""","""US02079K3059""","""02079K305""","""NASDAQ Global Select""","""NASDAQ""","""Internet Content & Information""","""https://www.abc.xyz""","""Alphabet Inc. provides various…","""Mr. Sundar Pichai""","""Communication Services""","""US""","""181269""","""650 253 0000""","""1600 Amphitheatre Parkway""","""Mountain View""","""CA""","""94043""",-26.11715,198.607146,"""https://images.financialmodeli…","""2004-08-19""",false,false,true,false,false
"""AMZN""",202.61,1.146,36387879,2130444150000,0.0,"""141.5-215.9""",-8.87,"""Amazon.com, Inc.""","""USD""","""0001018724""","""US0231351067""","""023135106""","""NASDAQ Global Select""","""NASDAQ""","""Specialty Retail""","""https://www.amazon.com""","""Amazon.com, Inc. engages in th…","""Mr. Andrew R. Jassy""","""Consumer Cyclical""","""US""","""1551000""","""206 266 1000""","""410 Terry Avenue North""","""Seattle""","""WA""","""98109-5210""",151.86378,50.746221,"""https://images.financialmodeli…","""1997-05-15""",false,false,true,false,false
"""TSLA""",320.72,2.295,87482998,1029530443200,0.0,"""138.8-358.64""",9.54,"""Tesla, Inc.""","""USD""","""0001318605""","""US88160R1014""","""88160R101""","""NASDAQ Global Select""","""NASDAQ""","""Auto - Manufacturers""","""https://www.tesla.com""","""Tesla, Inc. designs, develops,…","""Mr. Elon R. Musk""","""Consumer Cyclical""","""US""","""140473""","""512 516 8177""","""1 Tesla Road""","""Austin""","""TX""","""78725""",253.49773,67.222272,"""https://images.financialmodeli…","""2010-06-29""",false,false,true,false,false


In [26]:
df_company_infor.columns

['symbol',
 'price',
 'beta',
 'volAvg',
 'mktCap',
 'lastDiv',
 'range',
 'changes',
 'companyName',
 'currency',
 'cik',
 'isin',
 'cusip',
 'exchange',
 'exchangeShortName',
 'industry',
 'website',
 'description',
 'ceo',
 'sector',
 'country',
 'fullTimeEmployees',
 'phone',
 'address',
 'city',
 'state',
 'zip',
 'dcfDiff',
 'dcf',
 'image',
 'ipoDate',
 'defaultImage',
 'isEtf',
 'isActivelyTrading',
 'isAdr',
 'isFund']